In [2]:
import pandas as pd
physicians = sqlContext.read.csv("../DescriptiveSpark/physician.csv", sep="|", header=True)
patients = sqlContext.read.csv("../DescriptiveSpark/patient.csv", sep="|", header=True)

In [36]:
years = ['2009', '2011', '2013', '2015', '2017']
for year in years:
    df = sqlContext.read.parquet("../MonthlySpark/" + year + "01.parquet")
    df = df.filter(df.MONTH_ID == (year+"01"))
    quantiles = df.filter(df.DOSE > 90).stat.approxQuantile("DOSE",[0.25,0.5,0.75],0.0)
    cutoff = (quantiles[2]-quantiles[0]) * 1.5 + quantiles[2]
    outliers = df.filter(df.DOSE > cutoff)
    valid_data = df.filter(df.DOSE <= cutoff)
    
    outliers_with_physician = outliers.join(physicians, outliers.PROVIDER_ID == physicians.PROVIDER_ID)
    valid_data_with_physician = valid_data.join(physicians, valid_data.PROVIDER_ID == physicians.PROVIDER_ID)
    
    with pd.ExcelWriter('../outliers/' + year + 'PhysiciansOutlierReport.xlsx') as writer:

        outliers_with_physician.groupBy(outliers_with_physician.GENDER_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Outlier_Gender', index=False)
        outliers_with_physician.groupBy(outliers_with_physician.ST_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Outlier_State' ,index=False)
        outliers_with_physician.groupBy(outliers_with_physician.ZIP).count().toPandas()\
            .to_excel(writer,sheet_name='Outlier_Zip', index=False)
        outliers_with_physician.groupBy(outliers_with_physician.PRI_SPCL_CD, outliers_with_physician.PRI_SPCL_DESC).count().toPandas()\
            .to_excel(writer,sheet_name='Outlier_Speciality', index=False)

        valid_data_with_physician.groupBy(valid_data_with_physician.GENDER_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Valid_Gender', index=False)
        valid_data_with_physician.groupBy(valid_data_with_physician.ST_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Valid_State' ,index=False)
        valid_data_with_physician.groupBy(valid_data_with_physician.ZIP).count().toPandas()\
            .to_excel(writer,sheet_name='Valid_Zip', index=False)
        valid_data_with_physician.groupBy(valid_data_with_physician.PRI_SPCL_CD, valid_data_with_physician.PRI_SPCL_DESC).count().toPandas()\
            .to_excel(writer,sheet_name='Valid_Speciality', index=False)
    
    outliers_with_physician.unpersist()
    valid_data_with_physician.unpersist()
    del outliers_with_physician
    del valid_data_with_physician
    
    outliers_with_patient = outliers.join(patients, outliers.PATIENT_ID == patients.PATIENT_ID)
    valid_data_with_patient = valid_data.join(patients, valid_data.PATIENT_ID == patients.PATIENT_ID)
    
    with pd.ExcelWriter('../outliers/' + year + 'PatientsOutlierReport.xlsx') as writer:
        outliers_with_patient.groupBy(outliers_with_patient.PAT_GENDER_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Outlier_Gender', index=False)
        outliers_with_patient.groupBy(year - outliers_with_patient.PAT_BRTH_YR_NBR).count().toPandas()\
            .to_excel(writer, sheet_name='Outlier_Age', index=False)
        valid_data_with_patient.groupBy(valid_data_with_patient.PAT_GENDER_CD).count().toPandas()\
            .to_excel(writer, sheet_name='Valid_Gender', index=False)
        valid_data_with_patient.groupBy(year - valid_data_with_patient.PAT_BRTH_YR_NBR).count().toPandas()\
            .to_excel(writer, sheet_name='Valid_Age', index=False)
        
    outliers_with_patient.unpersist()
    valid_data_with_patient.unpersist()
    del outliers_with_patient
    del valid_data_with_patient

In [40]:
years = ['2009', '2011', '2013', '2015', '2017']
cutoffs = {}
for year in years:
    df = sqlContext.read.parquet("../MonthlySpark/" + year + "01.parquet")
    df = df.filter(df.MONTH_ID == (year+"01"))
    quantiles = df.filter(df.DOSE > 90).stat.approxQuantile("DOSE",[0.25,0.5,0.75],0.0)
    cutoff = (quantiles[2]-quantiles[0]) * 1.5 + quantiles[2]
    cutoff_p = df.filter(df.DOSE > cutoff).count() / df.count()
    cutoffs[year] = { 'cut': cutoff, 'p': cutoff_p }
    
print(cutoffs)

{'2009': {'cut': 292.01612903225805, 'p': 0.009820453104369546}, '2011': {'cut': 439.4428152492668, 'p': 0.004167635123040712}, '2013': {'cut': 405.11679644048945, 'p': 0.003180522734639475}, '2015': {'cut': 353.15020161290323, 'p': 0.003453995873414182}, '2017': {'cut': 294.55645161290323, 'p': 0.00434336882848287}}
